## Instance Struture

In [8]:
mutable struct Instance
    n :: Int64
    D
    l
    function Instance(s)
        D = zeros(1,1)
        nNodes = 0
        l = 0
        if s != "STDIN"
            open(s) do f
                line = 1
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                nNodes = parse(Int64, lineSplited[1])
                l = parse(Int64, lineSplited[2])
                D = zeros(nNodes,nNodes)

                while ! eof(f)
                    lineContent = readline(f)
                    lineSplited = split(lineContent, " ")
                    for col = 1:nNodes
                        value = parse(Float64, lineSplited[col])
                        D[line,col] = value
                    end

                    line += 1
                end
            end
        else
            line = 1
            lineContent = readline("STDIN")
            lineSplited = split(lineContent, " ")
            nNodes = parse(Int64, lineSplited[1])
            l = parse(Int64, lineSplited[2])
            D = zeros(nNodes,nNodes)

            while ! eof(f)
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                for col = 1:nNodes
                    value = parse(Float64, lineSplited[col])
                    D[line,col] = value
                end

                line += 1
            end
        end
        new(nNodes,D,l)
    end
end

## Solution Structure


In [9]:
mutable struct Solution
    π :: Array{Int64,1} 
    v :: Float64       
    Solution(s::Solution) = new(copy(s.π),s.v)
    Solution(π,v) = new(copy(π),v)
end
Base.copy(s::Solution) = Solution(s)

 ## Nearest Neighbor

In [10]:
function nearestNeighbor(I::Instance)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for k=2:I.n
      f=findall(free) 
        j=f[findmin(I.D[s.π[end],f])[2]] 
      push!(s.π,j) 
      free[j]=false
    end
    for lookahead=1:I.l 
        for i=1:I.n
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
    s
end

nearestNeighbor (generic function with 1 method)

## Random Nearest Neighbor

In [11]:
function randomNearestNeighbor(I::Instance,k)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for i=2:I.n
        f=findall(free)
        ncand=min(k,I.n-i+1)
        j=f[sortperm(I.D[s.π[end],f])[rand(1:ncand)]]
        push!(s.π,j)
        free[j]=false
    end
    for lookahead=1:I.l
        for i=1:I.n
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
    s
end

randomNearestNeighbor (generic function with 1 method)

## Local Search with Adjacent Nodes Swap

In [12]:
function calcIndex(index, n)
    result = 0
    if index <= 0
        result = n-(abs(index)%n)
    else
        result = (((index)-1)%(n))+1
    end
    result
end

function localSearchAdjacentNodesSwap(I::Instance, s₀::Solution)
    s = copy(s₀)
    i=1
    v=0.0
    while i<I.n
        i=2
        while i<I.n  
            v = s.v
            for l1=1:I.l+2
                index1 = calcIndex((i+1)-l1+1, I.n)
                for l2=1:I.l
                    index2 = calcIndex(index1+l2, I.n)
                    v -= I.D[s.π[index1],s.π[index2]]
                end
            end
            s.π[i],s.π[i+1] = s.π[i+1],s.π[i] # swap
            for l1=1:I.l+2
                index1 = calcIndex((i+1)-l1+1, I.n)
                for l2=1:I.l
                    index2 = calcIndex(index1+l2, I.n)
                    v += I.D[s.π[index1],s.π[index2]]
                end
            end
            s.π[i],s.π[i+1] = s.π[i+1],s.π[i] # swap again
            v<s.v && break ## first improvement
            i += 1
        end
        if i<I.n
            s.π[i],s.π[i+1] = s.π[i+1],s.π[i]
            s.v = v
        end
    end
    s
end

localSearchAdjacentCitiesSwap (generic function with 1 method)

## GRASP

In [18]:
I = Instance("instances/test72.txt")
b = nearestNeighbor(I)
println("Nearest Neighbor: $(b.v).")

nIterations = 100 
k = 3 

println("Local Search Adjacent Nodes Swap")
for i = 1:nIterations
    s = randomNearestNeighbor(I,k) 
    s = localSearchAdjacentNodesSwap(I,s)
    if s.v < b.v
        b = copy(s)
    end
    println("Iteration $i: $(b.v)\n")
end
println("Best solution $(b.v).")



Nearest Neighbor: 26.0.
Local Search Adjacent Cities Swap
2
[1, 5, 4, 7, 3, 6, 2]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 4, 5, 7, 3, 6, 2]
v: 24.0 | vok: 24.0 | s.v: 21.0
3
[1, 5, 4, 7, 3, 6, 2]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 5, 7, 4, 3, 6, 2]
v: 18.0 | vok: 18.0 | s.v: 21.0
2
[1, 5, 7, 4, 3, 6, 2]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 7, 5, 4, 3, 6, 2]
v: 19.0 | vok: 19.0 | s.v: 18.0
3
[1, 5, 7, 4, 3, 6, 2]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 5, 4, 7, 3, 6, 2]
v: 21.0 | vok: 21.0 | s.v: 18.0
4
[1, 5, 7, 4, 3, 6, 2]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 5, 7, 3, 4, 6, 2]
v: 21.0 | vok: 21.0 | s.v: 18.0
5
[1, 5, 7, 4, 3, 6, 2]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 5, 7, 4, 6, 3, 2]
v: 23.0 | vok: 23.0 | s.v: 18.0
6
[1, 5, 7, 4, 3, 6, 2]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 5, 7, 4, 3, 2, 6]
v: 21.0 | vok: 21.0 | s.v: 18.0
Iteration 1: 18.0

2
[1, 5, 3, 4, 6, 2, 7]
3 - 4
3 - 5
2 - 3

[1, 2, 6, 5, 4, 7, 3]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 2, 6, 5, 7, 4, 3]
v: 23.0 | vok: 23.0 | s.v: 20.0
6
[1, 2, 6, 5, 4, 7, 3]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 2, 6, 5, 4, 3, 7]
v: 20.0 | vok: 20.0 | s.v: 20.0
Iteration 7: 17.0

2
[1, 2, 3, 4, 7, 5, 6]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 3, 2, 4, 7, 5, 6]
v: 28.0 | vok: 28.0 | s.v: 26.0
3
[1, 2, 3, 4, 7, 5, 6]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 2, 4, 3, 7, 5, 6]
v: 23.0 | vok: 23.0 | s.v: 26.0
2
[1, 2, 4, 3, 7, 5, 6]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 4, 2, 3, 7, 5, 6]
v: 23.0 | vok: 23.0 | s.v: 23.0
3
[1, 2, 4, 3, 7, 5, 6]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 2, 3, 4, 7, 5, 6]
v: 26.0 | vok: 26.0 | s.v: 23.0
4
[1, 2, 4, 3, 7, 5, 6]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 2, 4, 7, 3, 5, 6]
v: 22.0 | vok: 22.0 | s.v: 23.0
2
[1, 2, 4, 7, 3, 5, 6]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 4, 2, 7, 3, 5, 6]
v: 22.0

5
[1, 4, 5, 6, 3, 2, 7]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 4, 5, 6, 2, 3, 7]
v: 20.0 | vok: 20.0 | s.v: 17.0
6
[1, 4, 5, 6, 3, 2, 7]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 4, 5, 6, 3, 7, 2]
v: 22.0 | vok: 22.0 | s.v: 17.0
Iteration 13: 17.0

2
[1, 2, 6, 5, 3, 4, 7]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 6, 2, 5, 3, 4, 7]
v: 19.0 | vok: 19.0 | s.v: 18.0
3
[1, 2, 6, 5, 3, 4, 7]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 2, 5, 6, 3, 4, 7]
v: 17.0 | vok: 17.0 | s.v: 18.0
2
[1, 2, 5, 6, 3, 4, 7]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 5, 2, 6, 3, 4, 7]
v: 17.0 | vok: 17.0 | s.v: 17.0
3
[1, 2, 5, 6, 3, 4, 7]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 2, 6, 5, 3, 4, 7]
v: 18.0 | vok: 18.0 | s.v: 17.0
4
[1, 2, 5, 6, 3, 4, 7]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 2, 5, 3, 6, 4, 7]
v: 20.0 | vok: 20.0 | s.v: 17.0
5
[1, 2, 5, 6, 3, 4, 7]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 2, 5, 6, 4, 3, 7]
v: 2

[1, 5, 2, 7, 3, 4, 6]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 2, 5, 7, 3, 4, 6]
v: 19.0 | vok: 19.0 | s.v: 19.0
3
[1, 5, 2, 7, 3, 4, 6]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 5, 7, 2, 3, 4, 6]
v: 21.0 | vok: 21.0 | s.v: 19.0
4
[1, 5, 2, 7, 3, 4, 6]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 5, 2, 3, 7, 4, 6]
v: 20.0 | vok: 20.0 | s.v: 19.0
5
[1, 5, 2, 7, 3, 4, 6]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 5, 2, 7, 4, 3, 6]
v: 25.0 | vok: 25.0 | s.v: 19.0
6
[1, 5, 2, 7, 3, 4, 6]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 5, 2, 7, 3, 6, 4]
v: 21.0 | vok: 21.0 | s.v: 19.0
Iteration 22: 17.0

2
[1, 4, 3, 5, 2, 6, 7]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 3, 4, 5, 2, 6, 7]
v: 25.0 | vok: 25.0 | s.v: 23.0
3
[1, 4, 3, 5, 2, 6, 7]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 4, 5, 3, 2, 6, 7]
v: 21.0 | vok: 21.0 | s.v: 23.0
2
[1, 4, 5, 3, 2, 6, 7]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 5, 4, 3, 2, 6, 7]
v: 24.

[1, 7, 3, 2, 6, 5, 4]
v: 20.0 | vok: 20.0 | s.v: 21.0
2
[1, 7, 3, 2, 6, 5, 4]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 3, 7, 2, 6, 5, 4]
v: 21.0 | vok: 21.0 | s.v: 20.0
3
[1, 7, 3, 2, 6, 5, 4]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 7, 2, 3, 6, 5, 4]
v: 17.0 | vok: 17.0 | s.v: 20.0
2
[1, 7, 2, 3, 6, 5, 4]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 2, 7, 3, 6, 5, 4]
v: 22.0 | vok: 22.0 | s.v: 17.0
3
[1, 7, 2, 3, 6, 5, 4]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 7, 3, 2, 6, 5, 4]
v: 20.0 | vok: 20.0 | s.v: 17.0
4
[1, 7, 2, 3, 6, 5, 4]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 7, 2, 6, 3, 5, 4]
v: 20.0 | vok: 20.0 | s.v: 17.0
5
[1, 7, 2, 3, 6, 5, 4]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 7, 2, 3, 5, 6, 4]
v: 18.0 | vok: 18.0 | s.v: 17.0
6
[1, 7, 2, 3, 6, 5, 4]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 7, 2, 3, 6, 4, 5]
v: 17.0 | vok: 17.0 | s.v: 17.0
Iteration 28: 17.0

2
[1, 4, 5, 6, 7, 2, 3]
3 - 4
3 - 5
2 - 3
2 

...

[1, 4, 3, 5, 7, 6, 2]
v: 26.0 | vok: 26.0 | s.v: 24.0
3
[1, 3, 4, 5, 7, 6, 2]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 3, 5, 4, 7, 6, 2]
v: 24.0 | vok: 24.0 | s.v: 24.0
4
[1, 3, 4, 5, 7, 6, 2]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 3, 4, 7, 5, 6, 2]
v: 23.0 | vok: 23.0 | s.v: 24.0
2
[1, 3, 4, 7, 5, 6, 2]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 4, 3, 7, 5, 6, 2]
v: 26.0 | vok: 26.0 | s.v: 23.0
3
[1, 3, 4, 7, 5, 6, 2]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 3, 7, 4, 5, 6, 2]
v: 20.0 | vok: 20.0 | s.v: 23.0
2
[1, 3, 7, 4, 5, 6, 2]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 7, 3, 4, 5, 6, 2]
v: 20.0 | vok: 20.0 | s.v: 20.0
3
[1, 3, 7, 4, 5, 6, 2]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 3, 4, 7, 5, 6, 2]
v: 23.0 | vok: 23.0 | s.v: 20.0
4
[1, 3, 7, 4, 5, 6, 2]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 3, 7, 5, 4, 6, 2]
v: 23.0 | vok: 23.0 | s.v: 20.0
5
[1, 3, 7, 4, 5, 6, 2]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 

[1, 3, 4, 5, 6, 2, 7]
v: 21.0 | vok: 21.0 | s.v: 20.0
5
[1, 3, 4, 6, 5, 2, 7]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 3, 4, 6, 2, 5, 7]
v: 23.0 | vok: 23.0 | s.v: 20.0
6
[1, 3, 4, 6, 5, 2, 7]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 3, 4, 6, 5, 7, 2]
v: 23.0 | vok: 23.0 | s.v: 20.0
Iteration 76: 17.0

2
[1, 5, 4, 6, 7, 2, 3]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 4, 5, 6, 7, 2, 3]
v: 22.0 | vok: 22.0 | s.v: 22.0
3
[1, 5, 4, 6, 7, 2, 3]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 5, 6, 4, 7, 2, 3]
v: 26.0 | vok: 26.0 | s.v: 22.0
4
[1, 5, 4, 6, 7, 2, 3]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 5, 4, 7, 6, 2, 3]
v: 24.0 | vok: 24.0 | s.v: 22.0
5
[1, 5, 4, 6, 7, 2, 3]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 5, 4, 6, 2, 7, 3]
v: 24.0 | vok: 24.0 | s.v: 22.0
6
[1, 5, 4, 6, 7, 2, 3]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 5, 4, 6, 7, 3, 2]
v: 23.0 | vok: 23.0 | s.v: 22.0
Iteration 77: 17.0

2
[1, 4, 6, 5, 3, 2, 7]


[1, 4, 6, 3, 2, 7, 5]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 4, 3, 6, 2, 7, 5]
v: 23.0 | vok: 23.0 | s.v: 18.0
4
[1, 4, 6, 3, 2, 7, 5]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 4, 6, 2, 3, 7, 5]
v: 21.0 | vok: 21.0 | s.v: 18.0
5
[1, 4, 6, 3, 2, 7, 5]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 4, 6, 3, 7, 2, 5]
v: 21.0 | vok: 21.0 | s.v: 18.0
6
[1, 4, 6, 3, 2, 7, 5]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 4, 6, 3, 2, 5, 7]
v: 19.0 | vok: 19.0 | s.v: 18.0
Iteration 84: 17.0

2
[1, 4, 5, 2, 3, 7, 6]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 5, 4, 2, 3, 7, 6]
v: 23.0 | vok: 23.0 | s.v: 23.0
3
[1, 4, 5, 2, 3, 7, 6]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 4, 2, 5, 3, 7, 6]
v: 23.0 | vok: 23.0 | s.v: 23.0
4
[1, 4, 5, 2, 3, 7, 6]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 4, 5, 3, 2, 7, 6]
v: 19.0 | vok: 19.0 | s.v: 23.0
2
[1, 4, 5, 3, 2, 7, 6]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 5, 4, 3, 2, 7, 6]
v: 22.

[1, 4, 5, 2, 3, 7, 6]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 4, 5, 3, 2, 7, 6]
v: 19.0 | vok: 19.0 | s.v: 23.0
2
[1, 4, 5, 3, 2, 7, 6]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 5, 4, 3, 2, 7, 6]
v: 22.0 | vok: 22.0 | s.v: 19.0
3
[1, 4, 5, 3, 2, 7, 6]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 4, 3, 5, 2, 7, 6]
v: 23.0 | vok: 23.0 | s.v: 19.0
4
[1, 4, 5, 3, 2, 7, 6]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 4, 5, 2, 3, 7, 6]
v: 23.0 | vok: 23.0 | s.v: 19.0
5
[1, 4, 5, 3, 2, 7, 6]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 4, 5, 3, 7, 2, 6]
v: 20.0 | vok: 20.0 | s.v: 19.0
6
[1, 4, 5, 3, 2, 7, 6]
7 - 1
7 - 2
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
[1, 4, 5, 3, 2, 6, 7]
v: 21.0 | vok: 21.0 | s.v: 19.0
Iteration 92: 17.0

2
[1, 5, 4, 6, 7, 3, 2]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 4, 5, 6, 7, 3, 2]
v: 26.0 | vok: 26.0 | s.v: 23.0
3
[1, 5, 4, 6, 7, 3, 2]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 5, 6, 4, 7, 3, 2]
v: 23.

3 - 4
3 - 5
2 - 3
2 - 4
[1, 3, 2, 6, 4, 5, 7]
v: 23.0 | vok: 23.0 | s.v: 24.0
2
[1, 3, 2, 6, 4, 5, 7]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 2, 3, 6, 4, 5, 7]
v: 21.0 | vok: 21.0 | s.v: 23.0
2
[1, 2, 3, 6, 4, 5, 7]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 3, 2, 6, 4, 5, 7]
v: 23.0 | vok: 23.0 | s.v: 21.0
3
[1, 2, 3, 6, 4, 5, 7]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 2, 6, 3, 4, 5, 7]
v: 19.0 | vok: 19.0 | s.v: 21.0
2
[1, 2, 6, 3, 4, 5, 7]
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
7 - 1
7 - 2
[1, 6, 2, 3, 4, 5, 7]
v: 24.0 | vok: 24.0 | s.v: 19.0
3
[1, 2, 6, 3, 4, 5, 7]
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
1 - 2
1 - 3
[1, 2, 3, 6, 4, 5, 7]
v: 21.0 | vok: 21.0 | s.v: 19.0
4
[1, 2, 6, 3, 4, 5, 7]
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
2 - 3
2 - 4
[1, 2, 6, 4, 3, 5, 7]
v: 21.0 | vok: 21.0 | s.v: 19.0
5
[1, 2, 6, 3, 4, 5, 7]
6 - 7
6 - 1
5 - 6
5 - 7
4 - 5
4 - 6
3 - 4
3 - 5
[1, 2, 6, 3, 5, 4, 7]
v: 19.0 | vok: 19.0 | s.v: 19.0
6
[1, 2, 6, 3, 4, 5, 7]
7 - 1
7 - 2
6 - 

In [37]:
function writeInEndOfFile(content, outputFile)
    output = open(outputFile,"a")
    write(output, content);
    close(output)
end
D = zeros(1,1)
nNodes = 0
l = 0
open("instances/rml7.txt") do f
    line = 1
    lineContent = readline(f)
    lineSplited = split(lineContent, " ")
    nNodes = parse(Int64, lineSplited[1])
    l = parse(Int64, lineSplited[2])
    D = zeros(nNodes,nNodes)
    println(nNodes,l)
    while ! eof(f)
        lineContent = readline(f)
        lineSplited = split(lineContent, " ")
        for col = 1:nNodes
            value = parse(Int64, lineSplited[col])
            D[line,col] = value
        end

        line += 1
    end
    
    output = open("rml7_rewrite.txt","a")

    write(output,"$(nNodes) $(l)\n")
    
    for i=1:nNodes
        for j=1:nNodes
            value = Int(D[i,j])
            write(output,"$(value) ")
        end
        write(output,"\n")
    end
    close(output)

    solution = [1, 163, 113, 117, 155, 156, 112, 120, 116, 153, 115, 111, 109, 110, 154, 106, 108, 107, 173, 174, 48, 47, 56, 49, 50, 71, 55, 91, 92, 53, 70, 69, 90, 78, 77, 76, 67, 75, 66, 60, 61, 59, 33, 3, 38, 34, 10, 22, 4, 37, 6, 8, 9, 30, 29, 20, 13, 21, 36, 5, 12, 11, 7, 19, 18, 17, 16, 14, 15, 28, 27, 26, 25, 31, 23, 24, 63, 62, 64, 65, 74, 84, 85, 86, 87, 80, 82, 81, 88, 101, 100, 103, 104, 102, 105, 213, 207, 206, 208, 209, 205, 200, 211, 210, 197, 204, 196, 194, 195, 99, 97, 95, 189, 98, 188, 184, 96, 186, 128, 185, 183, 79, 94, 93, 182, 181, 73, 57, 72, 68, 58, 54, 52, 51, 32, 45, 41, 46, 35, 40, 44, 2, 43, 177, 178, 180, 39, 42, 172, 175, 176, 179, 114, 122, 192, 118, 126, 187, 191, 119, 127, 123, 190, 129, 130, 131, 124, 135, 133, 125, 165, 121, 139, 166, 167, 164, 157, 168, 161, 160, 480, 481, 483, 485, 486, 409, 412, 406, 405, 159, 401, 407, 158, 400, 404, 425, 424, 423, 403, 328, 398, 419, 327, 418, 329, 330, 332, 325, 324, 331, 334, 344, 343, 345, 335, 420, 333, 346, 347, 349, 348, 336, 350, 337, 351, 352, 353, 354, 339, 355, 422, 356, 426, 340, 421, 342, 358, 357, 362, 359, 360, 363, 361, 364, 381, 440, 383, 441, 385, 386, 387, 370, 388, 449, 371, 372, 450, 376, 374, 375, 377, 390, 391, 389, 379, 392, 395, 397, 396, 394, 393, 373, 451, 368, 369, 452, 448, 367, 366, 365, 384, 443, 380, 382, 434, 444, 428, 442, 427, 433, 439, 435, 429, 432, 446, 447, 430, 436, 438, 467, 413, 445, 437, 468, 462, 470, 417, 461, 463, 460, 471, 459, 465, 464, 544, 543, 518, 517, 500, 519, 416, 516, 497, 515, 414, 499, 415, 494, 410, 498, 495, 493, 487, 492, 491, 408, 411, 338, 484, 402, 399, 488, 326, 482, 162, 496, 140, 141, 144, 501, 145, 502, 142, 146, 513, 514, 520, 510, 509, 521, 507, 522, 231, 508, 525, 232, 523, 530, 524, 532, 533, 529, 531, 526, 547, 527, 538, 534, 535, 548, 282, 287, 286, 285, 536, 283, 284, 537, 539, 540, 541, 554, 555, 558, 560, 559, 557, 561, 552, 556, 553, 475, 551, 549, 474, 546, 550, 473, 476, 478, 479, 472, 457, 456, 477, 453, 454, 455, 458, 469, 545, 542, 511, 505, 512, 504, 503, 490, 489, 148, 147, 228, 150, 143, 149, 137, 152, 217, 216, 151, 138, 215, 219, 218, 201, 198, 220, 203, 202, 221, 226, 223, 225, 224, 254, 259, 257, 258, 255, 222, 262, 241, 242, 245, 263, 229, 230, 238, 237, 236, 246, 266, 243, 264, 235, 267, 248, 239, 233, 250, 251, 249, 268, 271, 269, 252, 270, 273, 275, 274, 276, 278, 280, 289, 288, 272, 281, 291, 290, 295, 294, 299, 277, 293, 300, 301, 312, 297, 296, 307, 309, 298, 302, 306, 304, 303, 305, 313, 322, 321, 311, 320, 319, 308, 323, 314, 315, 292, 316, 317, 318, 310, 265, 279, 244, 261, 260, 240, 253, 256, 247, 227, 234, 528, 199, 212, 466, 170, 214, 136, 171, 134, 169, 431, 193, 89, 506, 132, 83, 378, 341] 

    sortedSolution = sort(copy(solution))

    for s in eachindex(sortedSolution)
        println("$(s) - $(sortedSolution[s])")
    end

    vok = 0
    for lookahead=1:l 
        for i=1:nNodes
            vok += D[solution[i],solution[((i+lookahead-1)%(nNodes))+1]]
        end
    end
    println("vok: $(vok) ")
end

56120
1 - 1
2 - 2
3 - 3
4 - 4
5 - 5
6 - 6
7 - 7
8 - 8
9 - 9
10 - 10
11 - 11
12 - 12
13 - 13
14 - 14
15 - 15
16 - 16
17 - 17
18 - 18
19 - 19
20 - 20
21 - 21
22 - 22
23 - 23
24 - 24
25 - 25
26 - 26
27 - 27
28 - 28
29 - 29
30 - 30
31 - 31
32 - 32
33 - 33
34 - 34
35 - 35
36 - 36
37 - 37
38 - 38
39 - 39
40 - 40
41 - 41
42 - 42
43 - 43
44 - 44
45 - 45
46 - 46
47 - 47
48 - 48
49 - 49
50 - 50
51 - 51
52 - 52
53 - 53
54 - 54
55 - 55
56 - 56
57 - 57
58 - 58
59 - 59
60 - 60
61 - 61
62 - 62
63 - 63
64 - 64
65 - 65
66 - 66
67 - 67
68 - 68
69 - 69
70 - 70
71 - 71
72 - 72
73 - 73
74 - 74
75 - 75
76 - 76
77 - 77
78 - 78
79 - 79
80 - 80
81 - 81
82 - 82
83 - 83
84 - 84
85 - 85
86 - 86
87 - 87
88 - 88
89 - 89
90 - 90
91 - 91
92 - 92
93 - 93
94 - 94
95 - 95
96 - 96
97 - 97
98 - 98
99 - 99
100 - 100
101 - 101
102 - 102
103 - 103
104 - 104
105 - 105
106 - 106
107 - 107
108 - 108
109 - 109
110 - 110
111 - 111
112 - 112
113 - 113
114 - 114
115 - 115
116 - 116
117 - 117
118 - 118
119 - 119
120 - 120
121 - 121
